In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [5]:
# Load the train-data CSV file
train_data = pd.read_csv('train-data.csv')

In [6]:
# Load the test-data CSV file
test_data = pd.read_csv('test-data.csv')

In [7]:
# Remove unnecessary columns (such as 'Unnamed: 0' and 'New_Price')
train_data = train_data.drop(['Unnamed: 0', 'New_Price'], axis=1)
test_data = test_data.drop(['Unnamed: 0', 'New_Price'], axis=1)

In [8]:
# Add a flag column to differentiate train and test datasets
train_data['Flag'] = 1
test_data['Flag'] = 0

In [9]:
# Combine train and test data for preprocessing
combined_data = pd.concat([train_data, test_data])

In [10]:
# Preprocess the data
combined_data['Mileage'] = combined_data['Mileage'].str.extract('(\d+\.\d+|\d+)').astype(float)
combined_data['Engine'] = combined_data['Engine'].str.extract('(\d+)').astype(float).fillna(0).astype(int)
combined_data['Power'] = combined_data['Power'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [11]:
# Convert categorical columns to numerical using LabelEncoder
label_encoder = LabelEncoder()
categorical_cols = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']
for col in categorical_cols:
    combined_data[col] = label_encoder.fit_transform(combined_data[col])

In [12]:
# Perform one-hot encoding on the combined dataset
onehot_encoder = OneHotEncoder(sparse=False)
encoded_data = onehot_encoder.fit_transform(combined_data)

In [13]:
# Split the combined data back into train and test datasets
X_train = encoded_data[combined_data['Flag'] == 1, :-1]
y_train = encoded_data[combined_data['Flag'] == 1, -1]
X_test = encoded_data[combined_data['Flag'] == 0, :-1]

In [14]:
# Fill missing values with 0
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)

In [15]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Build the ANN model with ReLU activation
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

In [17]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

In [18]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

Epoch 1/10
189/189 [==============================] - 2s 7ms/step - loss: 0.2070 - accuracy: 0.8481
Epoch 2/10
189/189 [==============================] - 1s 6ms/step - loss: 0.1918 - accuracy: 0.8540
Epoch 3/10
189/189 [==============================] - 1s 7ms/step - loss: 0.1168 - accuracy: 0.9329
Epoch 4/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0555 - accuracy: 0.9796
Epoch 5/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0252 - accuracy: 0.9925
Epoch 6/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0111 - accuracy: 0.9972
Epoch 7/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0068 - accuracy: 0.9982
Epoch 8/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0043 - accuracy: 0.9983
Epoch 9/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0033 - accuracy: 0.9987
Epoch 10/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0027 - accuracy: 0.9990

In [19]:
# Predict on the test data
predictions = model.predict(X_test_scaled)

In [20]:
# Print the predicted prices
print(predictions)

[[0.8862885 ]
 [0.9899852 ]
 [1.0078901 ]
 ...
 [0.960745  ]
 [0.97452664]
 [0.96465886]]
